<a href="https://colab.research.google.com/github/danidlrm/NPL/blob/main/Grid_search_%26_cross_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install contractions
! pipn install plotly_express

/bin/bash: line 1: pipn: command not found


In [ ]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/LIDCI/5/PROCESAMIENTO DE LENGUAJE NATURAL/SuicTweets (2).csv')
df.head()

,tweet,intention
0,my life is meaningless i just want to end my l...,1
1,muttering i wanna die to myself daily for a fe...,1
2,work slave i really feel like my only purpose ...,1
3,i did something on the 2 of october i overdose...,1
4,i feel like no one cares i just want to die ma...,1


In [ ]:
import contractions
import re
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')
stop_words_en = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
mapper = {
    1: 'SUICIDE',
    0: 'NOT SUICIDE'
}
df['label']=df['intention'].map(mapper)
df.head()

,tweet,intention,label
0,my life is meaningless i just want to end my l...,1,SUICIDE
1,muttering i wanna die to myself daily for a fe...,1,SUICIDE
2,work slave i really feel like my only purpose ...,1,SUICIDE
3,i did something on the 2 of october i overdose...,1,SUICIDE
4,i feel like no one cares i just want to die ma...,1,SUICIDE


In [ ]:
df['label'].value_counts()

,count
label,
NOT SUICIDE,5121
SUICIDE,3998


In [ ]:
from tqdm import tqdm
import contractions
tqdm.pandas()

def preprocesamiento_texto(texto):
  #Arreglar texto
  texto = contractions.fix(texto)
  #Hacer texto minusculo
  texto = texto.lower()
  #regex
  texto = re.sub(r'[^a-zA-Z\s]','', texto)
  #Tokenizar
  texto = word_tokenize(texto)
  #Quitar stopwords
  texto = [token for token in texto if token not in stop_words_en]
  #Lemmatizar
  texto = [lemmatizer.lemmatize(word) for word in texto]
  #juntar todo en string
  texto = ' '.join(texto)
  return texto

In [ ]:
df['clean_tweet']=df['tweet'].progress_apply(preprocesamiento_texto)
df.head()

100%|██████████| 9119/9119 [00:13<00:00, 697.68it/s] 


,tweet,intention,label,clean_tweet
0,my life is meaningless i just want to end my l...,1,SUICIDE,life meaningless want end life badly life comp...
1,muttering i wanna die to myself daily for a fe...,1,SUICIDE,muttering want die daily month feel worthless ...
2,work slave i really feel like my only purpose ...,1,SUICIDE,work slave really feel like purpose life make ...
3,i did something on the 2 of october i overdose...,1,SUICIDE,something october overdosed felt alone horribl...
4,i feel like no one cares i just want to die ma...,1,SUICIDE,feel like one care want die maybe feel less lo...


In [ ]:
 X = df['clean_tweet']
 y = df['intention']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    train_size =0.7,
                                                    random_state=101,
                                                    stratify=y)

In [ ]:
!pip install --upgrade scikit-learn

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [ ]:
pipeline = Pipeline ([
    ('tfidf',TfidfVectorizer()),
    ('model',RandomForestClassifier())
])

In [ ]:
parameters = {
    'tfidf__ngram_range':[(1,2),(2,3)],
    'model__n_estimators':[5,10,100],
    'model__criterion':['gini','entropy']
}

In [ ]:
grid = GridSearchCV(pipeline,
                    parameters,
                    cv=10,
                    n_jobs=-1,
                    verbose=5,
                    scoring='accuracy'
                    )

In [ ]:
grid.fit(X_train, y_train)

Fitting 10 folds for each of 12 candidates, totalling 120 fits


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('model', RandomForestClassifier())]),
             n_jobs=-1,
             param_grid={'model__criterion': ['gini', 'entropy'],
                         'model__n_estimators': [5, 10, 100],
                         'tfidf__ngram_range': [(1, 2), (2, 3)]},
             scoring='accuracy', verbose=5)

In [ ]:
grid.best_params_

{'model__criterion': 'gini',
 'model__n_estimators': 100,
 'tfidf__ngram_range': (1, 2)}

In [ ]:
grid.best_score_

0.8835982456914948